In [ ]:
# 🏀 Balldontlie Sync – Setup (Local Only)

from notebook_context import *  # gives you project_root, sys.path, etc.

import requests
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path

# Where we'll save the CSVs
data_dir = project_root / "data"
data_dir.mkdir(exist_ok=True)

# --- Balldontlie config ---
BASE_URL = "https://api.balldontlie.io/nba/v1"

# 🔑 IMPORTANT:
# 1. Paste your API key here
# 2. DO NOT commit this notebook to GitHub with the real key in it.
#    Before committing, either clear the key or move it to an env var.
BALLDONTLIE_API_KEY = "c5ae7df3-682e-450c-b47e-f7e91396379e"

def _bdl_headers():
    """Auth header for Balldontlie."""
    return {"Authorization": BALLDONTLIE_API_KEY}

def bdl_safe_get(endpoint: str, params: dict | None = None) -> dict:
    """
    Small helper to call Balldontlie and print what we're doing.
    Example: bdl_safe_get("teams"), bdl_safe_get("games", {"dates[]": "2025-11-26"})
    """
    if params is None:
        params = {}

    url = f"{BASE_URL}/{endpoint}"
    resp = requests.get(url, headers=_bdl_headers(), params=params, timeout=15)

    print("GET", resp.url, "->", resp.status_code)

    # Friendly error for the common case you already hit: 401
    if resp.status_code == 401:
        raise RuntimeError(
            "401 Unauthorized from Balldontlie.\n"
            "- Double-check your API key.\n"
            "- Or your current plan might not include this endpoint."
        )

    resp.raise_for_status()
    return resp.json()

print("Project root:", project_root)
print("Data dir:", data_dir)
print("Balldontlie base URL:", BASE_URL)


In [ ]:
# 2. Fetch & cache NBA teams → data/balldontlie_teams.csv

raw_teams = bdl_safe_get("teams", params={"per_page": 100})
teams = raw_teams.get("data", [])

print(f"Received {len(teams)} teams from Balldontlie")

teams_df = pd.json_normalize(teams)

# Keep just the useful columns for QEPC
keep_cols = [
    "id",
    "abbreviation",
    "full_name",
    "name",
    "city",
    "conference",
    "division",
]
teams_df = teams_df[keep_cols]

out_path = data_dir / "balldontlie_teams.csv"
teams_df.to_csv(out_path, index=False)

print("Saved teams to:", out_path)
display(teams_df.head())


In [ ]:
# 3. Fetch recent NBA games → data/balldontlie_games_recent.csv

# How far back / forward to fetch
DAYS_BACK = 7      # past 7 days
DAYS_FORWARD = 1   # today + tomorrow, for example

today = datetime.today().date()
dates = []

for i in range(DAYS_BACK, 0, -1):
    dates.append((today - timedelta(days=i)).isoformat())

for i in range(0, DAYS_FORWARD + 1):
    dates.append((today + timedelta(days=i)).isoformat())

print("Dates to fetch:", dates)

all_games = []

for d in dates:
    # docs example: /nba/v1/games?dates[]=2024-11-13
    js = bdl_safe_get("games", params={"dates[]": d, "per_page": 100})
    games = js.get("data", [])
    print(f"{d}: {len(games)} games")
    all_games.extend(games)

print("\nTotal games fetched:", len(all_games))

if all_games:
    games_df = pd.json_normalize(all_games)

    out_path = data_dir / "balldontlie_games_recent.csv"
    games_df.to_csv(out_path, index=False)

    print("Saved recent games to:", out_path)
    display(games_df.head())
else:
    print("⚠️ No games returned for the selected date range.")
